In [1]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
import re 
import pandas as pd
from pathlib import Path

In [2]:
filename = "Bexar_2010_SF1_Profile.pdf"
#open allows you to read the file.
pdfFileObj = open(filename,'rb')
#The pdfReader variable is a readable object that will be parsed.
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#Discerning the number of pages will allow us to parse through all the pages.
num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page.
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
#This if statement exists to check if the above library returned words. 
#It's done because PyPDF2 cannot read scanned files.
if text != "":
    text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text.
else:
    text = textract.process(fileurl, method='tesseract', language='eng')

#Now we have a text variable that contains all the text derived from our PDF file. 
#Type print(text) to see what it contains. It likely contains a lot of spaces, possibly junk such as '\n,' etc.

In [3]:
print(text)

Census 2010, Summary File 1                                        TABLE OF CONTENTS                                                        TOC Page Area Name: Bexar County                                                                                  Area Type: State-County (Summary Level: 050)State: TX  County: 029                                                                                                                                    +===================================================================================================================================================+|                                                                                                                                                   ||                            ==========  PAGE TITLE  ==========                                 == Page mnemonic and notes ==     == Selected in == ||                                  (All available pages)                                             

In [4]:
splits = text.split("===================================================================================================================================================")
pop_sum = splits[3]
pop_sum_table_split = pop_sum.split(" |")

new_table_list = []
for s in pop_sum_table_split:
    s = s.replace('|---------------------------------------------------------------------------------|', '')
    new_table_list.append(s)

# Separating odd and even index elements 
left_tables = [] 
right_tables = [] 
for i in range(0, len(new_table_list)): 
    if i % 2: 
        right_tables.append(new_table_list[i]) 
    else : 
        left_tables.append(new_table_list[i])

In [6]:
new_table_list

['+|P1/P15/P16/P17/P35/P36/P37. POPULATION SUMMARY                                  ',
 'P12/P13. POPULATION BY AGE AND SEX                              ',
 '|Universe: Persons, Households, Families                                         ',
 'Universe: Persons                                               ',
 '|                                                                                ',
 '                                                                ',
 '|Total Population                1,714,773  Total Families                416,356',
 '                 Both Sexes            Male             Female  ',
 '|Total Households                  608,931   Population in Families     1,387,901',
 '                Number  Pct.      Number  Pct.      Number  Pct.',
 '| Population in Households       1,672,772     Under 18 years             457,066',
 '                                                                ',
 '|   Under 18 years                 463,862     18 years and over   

In [7]:
left_tables

['+|P1/P15/P16/P17/P35/P36/P37. POPULATION SUMMARY                                  ',
 '|Universe: Persons, Households, Families                                         ',
 '|                                                                                ',
 '|Total Population                1,714,773  Total Families                416,356',
 '|Total Households                  608,931   Population in Families     1,387,901',
 '| Population in Households       1,672,772     Under 18 years             457,066',
 '|   Under 18 years                 463,862     18 years and over          930,835',
 '|   18 years and over            1,208,910   Average Family Size             3.33',
 '| Average Household Size              2.75     Under 18 years                1.10',
 '|   Under 18 years                    0.76     18 years and over             2.24',
 '|   18 years and over                 1.99                                       ',
 ' 15 to 17       76,775   4.5      39,359   4.7     

In [15]:
county = re.findall("[a-zA-Z]+\ ?[a-zA-Z]+ County", splits[0])[0]
county

'Bexar County'

In [140]:
left_tables

['+|P1/P15/P16/P17/P35/P36/P37. POPULATION SUMMARY                                  ',
 '|Universe: Persons, Households, Families                                         ',
 '|                                                                                ',
 '|Total Population                1,714,773  Total Families                416,356',
 '|Total Households                  608,931   Population in Families     1,387,901',
 '| Population in Households       1,672,772     Under 18 years             457,066',
 '|   Under 18 years                 463,862     18 years and over          930,835',
 '|   18 years and over            1,208,910   Average Family Size             3.33',
 '| Average Household Size              2.75     Under 18 years                1.10',
 '|   Under 18 years                    0.76     18 years and over             2.24',
 '|   18 years and over                 1.99                                       ',
 ' 15 to 17       76,775   4.5      39,359   4.7     

In [9]:
pop_sum = left_tables[0:10]
pop_sum_d = pop_sum[3:10]

pop_sum_clean_index = []

for s in pop_sum_d:
    s = s.replace("|", "")
    s = s.replace("Under 18", "Under Eightteen")
    s = s.replace("18 years", "Eighteen years")
    re_s = re.findall("[a-zA-Z ]+", s)
    for string in re_s:
        string = string.rstrip().lstrip()
        pop_sum_clean_index.append(string)

pop_sum_clean_num = []

for s in pop_sum_d:
    s = s.replace("|", "")
    s = s.replace("Under 18", "Under Eightteen")
    s = s.replace("18 years", "Eighteen years")
    num = re.findall("[0-9]?[0-9]?[0-9]?\,?[0-9]?[0-9]?[0-9]?\,?\.?[0-9]+", s) 
    for number in num:
        number = number.replace(",","")
        pop_sum_clean_num.append(float(number))


In [10]:
pop_sum_clean_index

['Total Population',
 'Total Families',
 'Total Households',
 'Population in Families',
 'Population in Households',
 'Under Eightteen years',
 'Under Eightteen years',
 'Eighteen years and over',
 'Eighteen years and over',
 'Average Family Size',
 'Average Household Size',
 'Under Eightteen years',
 'Under Eightteen years',
 'Eighteen years and over']

In [11]:
pop_sum_clean_num

[1714773.0,
 416356.0,
 608931.0,
 1387901.0,
 1672772.0,
 457066.0,
 463862.0,
 930835.0,
 1208910.0,
 3.33,
 2.75,
 1.1,
 0.76,
 2.24]

In [16]:
county = re.findall("[a-zA-Z]+\ ?[a-zA-Z]+ County", splits[0])[0][:-7]
pop_dict = {"county":pop_sum_clean_index,county:pop_sum_clean_num}
pop_df = pd.DataFrame(pop_dict)
pop_df = pop_df.T
pop_df.columns = pop_df.iloc[0]
pop_df = pop_df.iloc[1:]
pop_df = pop_df.reset_index()

pop_df.columns = ["County",'Total Population', 'Total Families', 'Total Households',
       'Population in Families', 'Population in Households',
       'Family: Under Eightteen years','House: Under Eightteen years',
       'Family: Eighteen years and over', 'House: Eighteen years and over',
       'Average Family Size', 'Average Household Size',
       'Family: Under Eightteen years', 'House: Under Eightteen years',
       'Family: Eighteen years and over']

In [18]:
pop_df.shape

(1, 15)

In [141]:
pop_sum = left_tables[0:10]
pop_sum_d = pop_sum[3:10]
pop_sum_d

['|Total Population                1,714,773  Total Families                416,356',
 '|Total Households                  608,931   Population in Families     1,387,901',
 '| Population in Households       1,672,772     Under 18 years             457,066',
 '|   Under 18 years                 463,862     18 years and over          930,835',
 '|   18 years and over            1,208,910   Average Family Size             3.33',
 '| Average Household Size              2.75     Under 18 years                1.10',
 '|   Under 18 years                    0.76     18 years and over             2.24']

In [142]:
pop_sum_clean_index = []

for s in pop_sum_d:
    s = s.replace("|", "")
    s = s.replace("Under 18", "Under Eightteen")
    s = s.replace("18 years", "Eighteen years")
    re_s = re.findall("[a-zA-Z ]+", s)
    for string in re_s:
        string = string.rstrip().lstrip()
        pop_sum_clean_index.append(string)

In [143]:
len(pop_sum_clean_index)

14

In [144]:
pop_sum_clean_num = []

for s in pop_sum_d:
    s = s.replace("|", "")
    s = s.replace("Under 18", "Under Eightteen")
    s = s.replace("18 years", "Eighteen years")
    num = re.findall("[0-9]?[0-9]?[0-9]?\,?[0-9]?[0-9]?[0-9]?\,?\.?[0-9]+", s)
    #print(num)
    for number in num:
        number = number.replace(",","")
        pop_sum_clean_num.append(float(number))
        
print(len(pop_sum_clean_num))
print(pop_sum_clean_num)

14
[1714773.0, 416356.0, 608931.0, 1387901.0, 1672772.0, 457066.0, 463862.0, 930835.0, 1208910.0, 3.33, 2.75, 1.1, 0.76, 2.24]


In [124]:
race_sum_clean_num

[34.0, 0.0, 2.0, 0.0, 0.0, 11.0, 0.0, 13.0, 33.0]

In [125]:
county = re.findall("[a-zA-Z]+ County", splits[0])[0]
pop_dict = {"county":race_sum_clean_index,county:race_sum_clean_num}
pop_df = pd.DataFrame(pop_dict)
pop_df = pop_df.T
pop_df.columns = pop_df.iloc[0]
pop_df = pop_df.iloc[1:]
pop_df = pop_df.reset_index()
pop_df = pop_df.rename(columns = {"index":"county"})
pop_df.reset_index(drop=True, inplace=True)

In [126]:
pop_df

county,county,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian/Other Pacific Islander alone,Some Other Race alone,Two or More Races,Hispanic or Latino (Any race),"White alone, not Hispanic"
0,Loving County,34,0,2,0,0,11,0,13,33


In [114]:
d = {'county': ["Bexar County"], 'fake data': [3]}
df = pd.DataFrame(data=d)
df

,county,fake data
0,Bexar County,3


In [115]:
df2 = df.merge(pop_df, left_on = "county", right_on = "county")

In [116]:
df2

,county,fake data,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian/Other Pacific Islander alone,Some Other Race alone,Two or More Races,Hispanic or Latino (Any race),"White alone, not Hispanic"
0,Bexar County,3,456741,48986,5135,13787,706,66765,16811,312063,227648
